First:
- go to the terminal
- go to the `4_tuning_networks` folder, use `cd` to get there, or `z` to get there faster.
- if you are planning on closing the laptop to let this run by itself, start a tmux session by giving the command `tmux`. You can name the session with `ctrl+b $`, leave the tmux with `ctrl+b` d, access it again with `tmux a -t 0` or replace the 0 with the name you gave it. See [tmux](https://github.com/tmux/tmux/wiki/Getting-Started) for more info.
- Activate the `.venv`, and run `python hypertune.py`
- In the hypertune.py file, a tune_dir is specified: `models/ray`. We will check the contents of that folder after the hypertune finished. You can also use tensorboard to check the results. 
- this will take some time. I clocked 1901 seconds (30 minutes) on the VM

 0.0992908 |           119 |            3 |     26 |         576.744  |    0 |   0.942187

In [34]:
from pathlib import Path

import os 
tune_dir = Path("C:/Users/Admin/ray_results")

tune_dir = Path(tune_dir)
tune_dir.exists()


True

In [35]:
tunelogs = [d for d in tune_dir.iterdir()]
tunelogs.sort()
latest = tunelogs[-1]
latest

WindowsPath('C:/Users/Admin/ray_results/train_2024-01-30_11-03-18')

In [36]:
from ray.tune import ExperimentAnalysis
import ray
# ray.init(ignore_reinit_error=False)


analysis = ExperimentAnalysis(latest)


So, we find some info:

In [40]:
analysis.results_df.columns


Index(['iterations', 'train_loss', 'test_loss', 'Accuracy', 'timestamp',
       'checkpoint_dir_name', 'done', 'training_iteration', 'date',
       'time_this_iter_s', 'time_total_s', 'pid', 'hostname', 'node_ip',
       'time_since_restore', 'iterations_since_restore', 'experiment_tag',
       'config/num_classes', 'config/tune_dir', 'config/hidden',
       'config/dropout', 'config/num_layers'],
      dtype='object')

Let's focus on the parameters we wanted to tune.

In [41]:
import plotly.express as px

plot = analysis.results_df
select = ["Accuracy", "config/hidden", "config/dropout", "config/num_layers"]
p = plot[select].reset_index().dropna()


Let's sort by accuracy

In [42]:
p.sort_values("Accuracy", inplace=True)

Make a parallel plot

In [43]:
px.parallel_coordinates(p, color="Accuracy")


Get the best trial

In [44]:
analysis.get_best_trial(metric="test_loss", mode="min")


train_7d586b81

The top ten

In [45]:
p[-10:]


,trial_id,Accuracy,config/hidden,config/dropout,config/num_layers
40,f76345e5,0.968403,23,0.352342,2
23,09d4b234,0.968750,124,0.371845,2
49,4ee07261,0.973958,109,0.363262,3
36,786e001d,0.976389,106,0.347596,2
37,e263c252,0.976736,95,0.304308,2
19,02137c66,0.976736,67,0.159658,2
38,e0a4fa66,0.979167,123,0.339919,2
35,b7b01803,0.979861,126,0.166648,2
17,edaa736b,0.979861,53,0.329921,2
13,7d586b81,0.980556,91,0.320268,2


Or the best config

In [46]:
analysis.get_best_config(metric="Accuracy", mode="max")


{'num_classes': 2,
 'tune_dir': WindowsPath('../../models/ray'),
 'hidden': 91,
 'dropout': 0.32026756711760473,
 'num_layers': 2}